In [14]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils


SELECTED_STOCK_PATH, _ = utils.main_processor(
    data_dir='..\picks',  # search project ..\data
    downloads_dir='',  # None searchs Downloads dir, '' omits search1
    downloads_limit=60,  # search the first 10 files
    clean_name_override=None,  # override filename
    start_file_pattern='2025-', # search for files starting with 'df_'
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\setuptools\\_vendor', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_w

**Available '2025-' files:**

- (1) `[DATA]` `2025-04-01_cluster_stats_df.parquet` <span style='color:#00ffff'>(0.01 MB, 2025-04-02 09:43)</span>

- (2) `[DATA]` `2025-04-01_detailed_clusters_df.parquet` <span style='color:#00ffff'>(0.05 MB, 2025-04-02 09:43)</span>

- (3) `[DATA]` `2025-04-01_portf.txt` <span style='color:#00ffff'>(0.10 MB, 2025-04-02 09:43)</span>

- (4) `[DATA]` `2025-04-01_selected_stocks.parquet` <span style='color:#00ffff'>(0.03 MB, 2025-04-02 09:43)</span>

- (5) `[DATA]` `2025-04-01_selected_stocks.tsv` <span style='color:#00ffff'>(0.01 MB, 2025-04-02 09:43)</span>


Input a number to select file (1-5)



    **Selected paths:**
    - Source: `..\picks\2025-04-01_selected_stocks.parquet`  
    - Destination: `..\picks\2025-04-01_selected_stocks_clean.parquet`
    

In [9]:
print(f'SELECTED_STOCK_PATH: {SELECTED_STOCK_PATH}')  # SOURCE_PATH

SELECTED_STOCK_PATH: ..\picks\2025-04-01_selected_stocks.parquet


In [12]:
date_str = utils.extract_date_from_string(SELECTED_STOCK_PATH)
print(f'date_str: {date_str}')

date_str: 2025-04-01


In [24]:
selected_stocks = pd.read_parquet(SELECTED_STOCK_PATH)
selected_stocks_n_weights = selected_stocks['Weight'].copy()
selected_stocks_n_weights

Ticker
PHYS    0.179265
COR     0.164450
MCK     0.136936
IAU     0.109283
GLDM    0.109218
EPD     0.107112
OGE     0.099870
ATO     0.093865
Name: Weight, dtype: float64

In [16]:
OHLCV_PATH, _ = utils.main_processor(
    data_dir='..\data',  # search project ..\data
    downloads_dir=None,  # None searchs Downloads dir, '' omits search1
    downloads_limit=60,  # search the first 10 files
    clean_name_override=None,  # override filename
    start_file_pattern=f'df_OHLCV_{date_str}', # search for files starting with 'df_'
)


<span style='color:#00ffff;font-weight:500'>[Downloads] Scanned latest 60 files • Found 1 'df_OHLCV_2025-04-01' matches</span>

**Available 'df_OHLCV_2025-04-01' files:**

- (1) `[DATA]` `df_OHLCV_2025-04-01_clean.parquet` <span style='color:#00ffff'>(14.60 MB, 2025-04-01 17:10)</span>

- (2) `[DOWNLOADS]` `df_OHLCV_2025-04-01.parquet` <span style='color:#00ffff'>(14.69 MB, 2025-04-01 16:25)</span>


Input a number to select file (1-2)



    **Selected paths:**
    - Source: `..\data\df_OHLCV_2025-04-01_clean.parquet`  
    - Destination: `..\data\df_OHLCV_2025-04-01_clean_clean.parquet`
    

In [19]:
OHLCV = pd.read_parquet(OHLCV_PATH)
OHLCV.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('TBIL', Timestamp('2025-04-01 00:00:00')) to ('IEI', Timestamp('2024-04-02 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       386038 non-null  float64
 1   High       386038 non-null  float64
 2   Low        386038 non-null  float64
 3   Close      386038 non-null  float64
 4   Adj Close  386038 non-null  float64
 5   Volume     386038 non-null  Int64  
 6   Adj Open   386038 non-null  float64
 7   Adj High   386038 non-null  float64
 8   Adj Low    386038 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.4+ MB


In [30]:
OHLCV.loc['SPY'].index.name

'Date'

In [42]:
import pandas as pd

def create_performance_df(date_str, selected_stocks_n_weights, OHLCV):
    """
    Create a DataFrame tracking performance of stock selections.
    
    Parameters:
    - date_str: str, the date when stocks were selected (format 'YYYY-MM-DD')
    - selected_stocks_n_weights: pd.Series with tickers as index and weights as values
    - OHLCV: pd.DataFrame with historical stock prices (must contain 'Adj Close' and be multi-indexed by date and ticker)
    
    Returns:
    - pd.DataFrame with performance tracking information
    """
    
    # Convert date_str to datetime
    selection_date = pd.to_datetime(date_str)
    
    # Get all unique dates from OHLCV
    # all_dates = OHLCV.index.get_level_values('date').unique()
    all_dates = OHLCV.index.get_level_values('Date').unique()    
    all_dates = all_dates.sort_values()
    
    # Find the next trading date after selection_date (buy date)
    buy_date = all_dates[all_dates > selection_date].min()
    
    # Initialize performance DataFrame
    performance_data = []
    
    # Get the tickers and weights
    selected_tickers = selected_stocks_n_weights.index.tolist()
    weights = selected_stocks_n_weights.values.tolist()
    
    for ticker, weight in zip(selected_tickers, weights):
        # Get buy price (Adj_close on buy_date)
        try:
            # buy_price = OHLCV.loc[(buy_date, ticker), 'Adj Close']
            buy_price = OHLCV.loc[(ticker, buy_date), 'Adj Close']
        except KeyError:
            print(f"Could not find buy price for {ticker} on {buy_date}")
            continue
        
        # Find when this ticker was removed from the portfolio
        # This would depend on your logic - here I'm assuming you have a way to track
        # when stocks are removed from selected_stocks_n_weights in subsequent periods
        
        # For demonstration, let's assume we sell after a fixed holding period (e.g., 1 month)
        # In practice, you'd need to track when the stock is removed from selected_stocks_n_weights
        sell_date = all_dates[all_dates > buy_date].min() + pd.DateOffset(months=1)
        sell_date = min(sell_date, all_dates[-1])  # Don't exceed available data
        
        try:
            sell_price = OHLCV.loc[(ticker, sell_date), 'Adj Close']
        except KeyError:
            print(f"Could not find sell price for {ticker} on {sell_date}")
            continue
        
        # Calculate return
        return_pct = (sell_price - buy_price) / buy_price * 100
        dollar_return = sell_price - buy_price
        
        performance_data.append({
            'Ticker': ticker,
            'Weight': weight,
            'Selection_Date': selection_date,
            'Buy_Date': buy_date,
            'Buy_Price': buy_price,
            'Sell_Date': sell_date,
            'Sell_Price': sell_price,
            'Return_Pct': return_pct,
            'Dollar_Return': dollar_return,
            'Holding_Period': (sell_date - buy_date).days
        })
    
    performance_df = pd.DataFrame(performance_data)
    
    return performance_df

# Example usage:
# OHLCV should be a DataFrame with MultiIndex (date, ticker) and 'Adj Close' column
performance_df = create_performance_df('2025-01-31', selected_stocks_n_weights, OHLCV)

In [43]:
performance_df

,Ticker,Weight,Selection_Date,Buy_Date,Buy_Price,Sell_Date,Sell_Price,Return_Pct,Dollar_Return,Holding_Period
0,PHYS,0.179265,2025-01-31,2025-02-03,21.80,2025-03-04,22.62,3.761468,0.82,29
1,COR,0.164450,2025-01-31,2025-02-03,253.86,2025-03-04,253.28,-0.228472,-0.58,29
2,MCK,0.136936,2025-01-31,2025-02-03,599.15,2025-03-04,643.69,7.433865,44.54,29
3,IAU,0.109283,2025-01-31,2025-02-03,53.17,2025-03-04,55.04,3.517021,1.87,29
4,GLDM,0.109218,2025-01-31,2025-02-03,55.81,2025-03-04,57.79,3.547751,1.98,29
5,EPD,0.107112,2025-01-31,2025-02-03,33.13,2025-03-04,33.32,0.573498,0.19,29
6,OGE,0.099870,2025-01-31,2025-02-03,42.30,2025-03-04,45.12,6.666667,2.82,29
7,ATO,0.093865,2025-01-31,2025-02-03,142.57,2025-03-04,149.75,5.036123,7.18,29
